In [7]:
import pandas as pd
import numpy as np
import os
from ydata_profiling import ProfileReport
import utils

In [8]:
with open("bdd_profilweb.csv", "r",encoding="iso-8859-1") as file:
    file_contents = file.read()

In [9]:
file_contents=file_contents.replace('Ã©','é')
file_contents=file_contents.replace('Ã¨','è')
rows=file_contents.split('\n')
header=rows[0].split(';')
data=[i.split(';') for i in rows[1:]]
df=pd.DataFrame(data,columns=header)

In [39]:
df=df.iloc[:50000]
df_clean=df.copy()
#df_clean.replace('?',,inplace=True) Non ce sont les visiteurs non clients
#df_clean.replace('manquant',np.nan,inplace=True)
df_clean=df_clean.replace('NR',np.nan)
df_clean=df_clean.replace('<18 ans',np.nan)
df_clean=df_clean.drop(df_clean[df_clean['revfyr']=='manquant'][df_clean['montant_credit']=='?'][df_clean['sitfam']=='NR'].index,axis=0)
df_clean=df_clean=df_clean.drop('activite',axis=1)
df_clean=df_clean=df_clean.drop('dept',axis=1)
df_clean=df_clean.dropna().reset_index()
df_clean=df.drop('activite',axis=1)

C:\Users\lenovo\AppData\Local\Temp\ipykernel_46808\1430975954.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_clean=df_clean.drop(df_clean[df_clean['revfyr']=='manquant'][df_clean['montant_credit']=='?'][df_clean['sitfam']=='NR'].index,axis=0)
C:\Users\lenovo\AppData\Local\Temp\ipykernel_46808\1430975954.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_clean=df_clean.drop(df_clean[df_clean['revfyr']=='manquant'][df_clean['montant_credit']=='?'][df_clean['sitfam']=='NR'].index,axis=0)


In [40]:
Regroupement={'<4':['3', '4', '1', '2'],
              '5-10':['5-6', '7-10'], 
              '>11':['11-20', '>20'],
              'Referencement':['Moteurs', 'Sites du Portail', 'Sites affluents'], 
              'Recherche directe':['Accès Direct', 'Notoriété'], 
              'Marketing':['Webmails', 'Email marketing','Affiliation et partenaires', 'Liens Sponsorisés','Publicité'],
              '>6000':['5.]6000e- 8000e]', '6.]8000e- 12000e]', '7.>12000e'],
              '<12m':['6m-12m', '- de 6m'],
              '++':['++', '+'],
              'proprietaire':['proprietaire', 'Accession pro'],
              'Locataire':['Locataire', 'Loge chez employeur', 'Autres', 'Loge chez parents'],
              'Divorce - separe - Veuf':['Divorce - separe', 'Veuf'],
              '>66':['66-75 ans', '>=76 ans'], 
              '>4000':['4000 a 4999', '5000 et plus']}

In [41]:
df_clean

,num_cli,connexion_mois,source_web,typ_client,montant_credit,anciennete,utilisation_credit,logmt,sitfam,dept,tr_age,revfyr,assurance
0,1,3,Moteurs,Reserve d'argent,5.]6000e- 8000e],>24 mois,+++,Locataire,Marie - Union libre,Lorraine,26-35 ans,2000 a 2999,credit assure
1,2,4,Liens Sponsorisés,Reserve d'argent,2.]1000e- 3000e],>24 mois,+++,Locataire,Celibataire,Aquitaine,26-35 ans,1000 a 1499,credit assure
2,3,1,Moteurs,Reserve d'argent,4.]3000e- 6000e],>24 mois,+++,Accession pro,Marie - Union libre,Lorraine,26-35 ans,2000 a 2999,credit non assure
3,4,4,Moteurs,Reserve d'argent,4.]3000e- 6000e],6m-12m,+++,Locataire,Celibataire,Auvergne,18-25 ans,1500 a 1999,credit assure
4,5,5-6,Accès Direct,Reserve d'argent,6.]8000e- 12000e],>24 mois,++,Loge chez employeur,Marie - Union libre,Haute-Normandie,36-45 ans,3000 a 3999,credit non assure
...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,49996,2,Accès Direct,Reserve d'argent,2.]1000e- 3000e],6m-12m,+++,Locataire,Marie - Union libre,Alsace,46-55 ans,3000 a 3999,credit non assure
49996,49997,2,Accès Direct,Reserve d'argent,5.]6000e- 8000e],>24 mois,+++,Loge chez parents,Celibataire,Poitou-Charentes,26-35 ans,1000 a 1499,credit assure
49997,49998,5-6,Moteurs,Reserve d'argent,6.]8000e- 12000e],>24 mois,+++,Locataire,Divorce - separe,Provence-Alpes-Côte-Azur,56-65 ans,1500 a 1999,credit assure
49998,49999,3,Accès Direct,Credit voiture,4.]3000e- 6000e],>24 mois,+++,Locataire,Marie - Union libre,Haute-Normandie,26-35 ans,3000 a 3999,credit non assure


In [64]:
def regroupement(dataframe,dico):
    for k,v in dico.items():
        print(k,v)
        dataframe=dataframe.replace(v,k)
    return dataframe

df_clean_regroup=regroupement(df_clean[df_clean.columns[1:]],Regroupement)[:-1]

<4 ['3', '4', '1', '2']
5-10 ['5-6', '7-10']
>11 ['11-20', '>20']
Referencement ['Moteurs', 'Sites du Portail', 'Sites affluents']
Recherche directe ['Accès Direct', 'Notoriété']
Marketing ['Webmails', 'Email marketing', 'Affiliation et partenaires', 'Liens Sponsorisés', 'Publicité']
>6000 ['5.]6000e- 8000e]', '6.]8000e- 12000e]', '7.>12000e']
<12m ['6m-12m', '- de 6m']
++ ['++', '+']
proprietaire ['proprietaire', 'Accession pro']
Locataire ['Locataire', 'Loge chez employeur', 'Autres', 'Loge chez parents']
Divorce - separe - Veuf ['Divorce - separe', 'Veuf']
>66 ['66-75 ans', '>=76 ans']
>4000 ['4000 a 4999', '5000 et plus']


In [65]:
col=['num_cli']
for i in df_clean_regroup.columns[1:]:
    col.append(i)
col

['num_cli',
 'connexion_mois',
 'source_web',
 'typ_client',
 'montant_credit',
 'anciennete',
 'utilisation_credit',
 'logmt',
 'sitfam',
 'dept',
 'tr_age',
 'revfyr',
 'assurance']

In [66]:
df_clean_regroup=df_clean_regroup[df_clean_regroup.columns[1:]]
df_clean_regroup=df_clean_regroup.reset_index()
df_clean_regroup.columns=col
df_clean_regroup['num_cli']=df_clean_regroup['num_cli']+1

In [16]:
df_clean_regroup=df_clean_regroup.reset_index(drop=True)
df_clean_regroup.to_csv('df_clean_regroup.csv',sep=';')

In [67]:
ind_nr=list()
ind_snr=list()
for i in range(len(df_clean_regroup)):
    if '?' in list(df_clean_regroup.iloc[i]):
        ind_nr.append(i)
    else:
        ind_snr.append(i)

In [73]:
df_clean_regroup.iloc[ind_nr].to_csv('data_clean_regroup_non_cli.csv',sep=';')

In [74]:
df_clean_regroup.iloc[ind_snr].to_csv('data_clean_regroup_cli.csv',sep=';')

In [72]:
df_clean_regroup.iloc[ind_snr]

,num_cli,connexion_mois,source_web,typ_client,montant_credit,anciennete,utilisation_credit,logmt,sitfam,dept,tr_age,revfyr,assurance
0,1,<4,Referencement,Reserve d'argent,>6000,>24 mois,+++,Locataire,Marie - Union libre,Lorraine,26-35 ans,2000 a 2999,credit assure
1,2,<4,Marketing,Reserve d'argent,2.]1000e- 3000e],>24 mois,+++,Locataire,Celibataire,Aquitaine,26-35 ans,1000 a 1499,credit assure
2,3,<4,Referencement,Reserve d'argent,4.]3000e- 6000e],>24 mois,+++,proprietaire,Marie - Union libre,Lorraine,26-35 ans,2000 a 2999,credit non assure
3,4,<4,Referencement,Reserve d'argent,4.]3000e- 6000e],<12m,+++,Locataire,Celibataire,Auvergne,18-25 ans,1500 a 1999,credit assure
4,5,5-10,Recherche directe,Reserve d'argent,>6000,>24 mois,++,Locataire,Marie - Union libre,Haute-Normandie,36-45 ans,3000 a 3999,credit non assure
...,...,...,...,...,...,...,...,...,...,...,...,...,...
49994,49995,<4,Marketing,Reserve d'argent,4.]3000e- 6000e],>24 mois,+++,Locataire,Divorce - separe - Veuf,Bourgogne,46-55 ans,1000 a 1499,credit assure
49995,49996,<4,Recherche directe,Reserve d'argent,2.]1000e- 3000e],<12m,+++,Locataire,Marie - Union libre,Alsace,46-55 ans,3000 a 3999,credit non assure
49996,49997,<4,Recherche directe,Reserve d'argent,>6000,>24 mois,+++,Locataire,Celibataire,Poitou-Charentes,26-35 ans,1000 a 1499,credit assure
49997,49998,5-10,Referencement,Reserve d'argent,>6000,>24 mois,+++,Locataire,Divorce - separe - Veuf,Provence-Alpes-Côte-Azur,56-65 ans,1500 a 1999,credit assure
